# Setup

In [4]:
import nengo
import numpy as np

#import pytry
import scipy
#import nni

from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

from datetime import datetime
import sys,os

import optuna

In [9]:
class arm:
    def __init__(self, l, m, dt):
        self.l0 = l[0]
        self.l1 = l[1]
        self.l2 = l[2]
        
        self.m0 = m[0]
        self.m1 = m[1]
        self.m2 = m[2]
        
        #self.theta = theta
        #self.thetaDot = thetaDot
        self.dt = dt
    def __call__(self,t,inputs): #inputs = [theta,theta_dot,U] -> [0:3,3:6,6:9] 
        theta_dot = np.zeros(6)
        JofXwrtQ3 = vJq(inputs[0:3],[self.l0,self.l1,self.l2]);
        JofXwrtQ2 = vJq(inputs[0:3],[self.l0,self.l1, 0]);
        JofXwrtQ1 = vJq(inputs[0:3],[self.l0, 0, 0]);

        Fg = np.matrix([0, -9.8, 0]).T;
        Fqg1 = JofXwrtQ1.T * Fg * self.m0;
        Fqg2 = JofXwrtQ2.T * Fg * self.m1;
        Fqg3 = JofXwrtQ3.T * Fg * self.m2;
        Fqg = Fqg1+Fqg2+Fqg3;
        
        #Find net torque
        Fqnet = Fqg + np.matrix(inputs[6:9]).T;
        
        #Get accerlation and velocity
        # Need joint limit check 
        theta_dot[3] = inputs[3] + self.dt*Fqnet.item(0)/(self.m0+self.m1+self.m2)
        theta_dot[4] = inputs[4] + self.dt*Fqnet.item(1)/(self.m1+self.m2)
        theta_dot[5] = inputs[5] + self.dt*Fqnet.item(2)/(self.m2)

        theta_dot[0] = inputs[0] + dt * inputs[3]
        theta_dot[1] = inputs[1] + dt * inputs[4]
        theta_dot[2] = inputs[2] + dt * inputs[5]
        
        return np.array(theta_dot)
def vJq(Theta,L):
    J = np.matrix(
            ([(-L[0]*np.sin(Theta[0])-L[1]*np.sin(Theta[0]+Theta[1])-L[2]*np.sin(Theta[0]+Theta[1]+Theta[2])), (-L[1]*np.sin(Theta[0]+Theta[1])-L[2]*np.sin(Theta[0]+Theta[1]+Theta[2])), (-L[2]*np.sin(Theta[0]+Theta[1]+Theta[2]))],
            [(L[0]*np.cos(Theta[0])+L[1]*np.cos(Theta[0]+Theta[1])+L[2]*np.cos(Theta[0]+Theta[1]+Theta[2])), (L[1]*np.cos(Theta[0]+Theta[1])+L[2]*np.cos(Theta[0]+Theta[1]+Theta[2])), (L[2]*np.cos(Theta[0]+Theta[1]+Theta[2]))],
            [0, 0, 0]))
    return J




# Sim Parameters

In [70]:

#Sim parameters
sim_time = 40

#Model parameters
num_neurons = 1000
sim_synapse = 0
model_neurons = nengo.LIF()

#Sys parameters
dt = 0.001
l = [0.1,0.1,0.1]
m = [0.1,0.1,0.1]
x_mean = [0,0,0,0]
limits = [0.3,0.3,-1,1]
ks = 129.41716480250645
lam = 0.8094360625273509


## No Adaptive

In [ ]:
def objective(trial):
    ks = trial.suggest_float("ks", 50, 200)
    lam = trial.suggest_float('lam',0.5,1)
    model, state_probe, reference_probe = create_model_no_adapt(ks,lam):
    with nengo.Simulator(model,dt = dt) as sim:
        sim.run(sim_time)
    ref_signal = sim.data[reference_probe]
    state_signal = -sim.data[state_probe]
    return np.sqrt(np.mean((ref_signal-state_signal)**2))
study = optuna.create_study()
study.optimize(objective, n_trials=200)

study.best_params  






## Random Adaptive

In [75]:
def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-8, 1e-5)
    model, state_probe, reference_probe = create_model_rand_adapt(learning_rate)
    with nengo.Simulator(model,dt = dt) as sim:
        sim.run(sim_time)
    ref_signal = sim.data[reference_probe]
    state_signal = -sim.data[state_probe]
    return np.sqrt(np.mean((ref_signal-state_signal)**2))
ks = 129.41716480250645
lam = 0.8094360625273509

study = optuna.create_study()
study.optimize(objective, n_trials=20)

study.best_params  






[I 2024-10-02 15:52:02,514] A new study created in memory with name: no-name-e9b39297-748e-4fe7-a505-01da014d8d60


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

[I 2024-10-02 15:52:33,361] Trial 0 finished with value: 0.06109204723226358 and parameters: {'learning_rate': 3.647820031757562e-06}. Best is trial 0 with value: 0.06109204723226358.


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

...

[I 2024-10-02 16:02:33,138] Trial 18 finished with value: 0.05180201397572734 and parameters: {'learning_rate': 2.615987524148069e-06}. Best is trial 6 with value: 0.045270547178237336.


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

[I 2024-10-02 16:03:07,245] Trial 19 finished with value: 0.04511335014180429 and parameters: {'learning_rate': 9.476087912119756e-07}. Best is trial 19 with value: 0.04511335014180429.


{'learning_rate': 9.476087912119756e-07}

## Selected Adaptive

In [76]:
def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-8, 1e-5)
    rho_specified = trial.suggest_float('rho_specified',0,1)
    gain = trial.suggest_float('gain',50,250)
    model, state_probe, reference_probe = create_model_proj_adapt(learning_rate,rho_specified,gain)
    with nengo.Simulator(model,dt = dt) as sim:
        sim.run(sim_time)
    ref_signal = sim.data[reference_probe]
    state_signal = -sim.data[state_probe]
    return np.sqrt(np.mean((ref_signal-state_signal)**2))
ks = 129.41716480250645
lam = 0.8094360625273509

study = optuna.create_study()
study.optimize(objective, n_trials=20)

study.best_params  






[I 2024-10-02 16:03:07,252] A new study created in memory with name: no-name-d0d6c2d9-c180-4509-8b78-c77a9d03a143


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

[I 2024-10-02 16:03:43,234] Trial 0 finished with value: 0.0658822188790086 and parameters: {'learning_rate': 1.6970121016147341e-06, 'rho_specified': 0.33611898889905134, 'gain': 168.36019297804853}. Best is trial 0 with value: 0.0658822188790086.


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

...

[I 2024-10-02 16:14:39,095] Trial 18 finished with value: 0.07715796799017503 and parameters: {'learning_rate': 3.0764475476659145e-06, 'rho_specified': 0.48916480035678567, 'gain': 174.28690513357222}. Best is trial 7 with value: 0.04364414216909527.


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

[I 2024-10-02 16:15:15,453] Trial 19 finished with value: 0.08391789029446485 and parameters: {'learning_rate': 8.415822122488191e-06, 'rho_specified': 0.2647032051724897, 'gain': 216.00175037095184}. Best is trial 7 with value: 0.04364414216909527.


{'learning_rate': 1.632674310846433e-07,
 'rho_specified': 0.625007481918219,
 'gain': 184.97387158155766}

## SSP Adaptive

In [77]:
def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-8, 1e-5)
    rho_specified = trial.suggest_float('rho_specified',0,1)
    gain = trial.suggest_float('gain',50,250)
    model, state_probe, reference_probe = create_model_SSP_adapt(learning_rate,rho_specified,gain)
    with nengo.Simulator(model,dt = dt) as sim:
        sim.run(sim_time)
    ref_signal = sim.data[reference_probe]
    state_signal = -sim.data[state_probe]
    return np.sqrt(np.mean((ref_signal-state_signal)**2))
ks = 129.41716480250645
lam = 0.8094360625273509

#encoders_type = 'random'
encoders_type = 'place-cells'
psd_sampling = 'gaussian'
domain_ranges_ = np.array([[-0.3,0.3],[-1.,1.],[-0.3,0.3],[-1.,1.]])
ssp_dim = 512
length_scale = np.asarray([[0.82,5.31,0.5,5]])

study = optuna.create_study()
study.optimize(objective, n_trials=20)
study.best_params  






[I 2024-10-02 16:15:15,462] A new study created in memory with name: no-name-89eec995-f40a-4619-ba97-b6b23c1f45cb


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

[I 2024-10-02 16:15:53,532] Trial 0 finished with value: 0.12005202408440131 and parameters: {'learning_rate': 3.929232725190017e-06, 'rho_specified': 0.6838034183440349, 'gain': 191.03734801235558}. Best is trial 0 with value: 0.12005202408440131.


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

...

[I 2024-10-02 16:28:21,601] Trial 18 finished with value: 0.11889582614410647 and parameters: {'learning_rate': 8.823582714558269e-06, 'rho_specified': 0.7729659108284045, 'gain': 117.1674041583212}. Best is trial 10 with value: 0.11669062676649736.


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc['progress']['progress_bar'] = 'nengo.utils.progress.TerminalProgressBar'`.

[I 2024-10-02 16:29:07,440] Trial 19 finished with value: 0.11731203777369538 and parameters: {'learning_rate': 7.028282560446665e-06, 'rho_specified': 0.267182175832955, 'gain': 165.21780680043548}. Best is trial 10 with value: 0.11669062676649736.


{'learning_rate': 9.600220700967684e-06,
 'rho_specified': 0.016674121848544776,
 'gain': 54.04610584169457}

# Model Functions

## No adaptive

In [63]:
def create_model_no_adapt(ks,lam):
    
    model = nengo.Network()
    with model:
        #Functions
        def Sliding_Mode(e): #e = [ex, ey, edotx, edoty] -> u = [ux,uy]
            s = e[0:2] + lam*e[2:4]
            g_x = m[2]
            return (1/g_x)*(ks*s + lam*e[2:4]) #u = [ux,uy]
            
        def qTx(Q): #Takes joint inputs and converts to end effector
            # Q = [q1,q2,q3,qdot1,qdot2,qdot3] -> p = [px, py, pdotx, pdoty]
            
            p0 = np.array([0,0])
            
            p1 = p0 + np.array([l[0]*np.cos(Q[0]),l[0]*np.sin(Q[0])])
            p2 = p1 + np.array([l[0]*np.cos(Q[0]+Q[1]),l[0]*np.sin(Q[0]+Q[1])])
            p3 = p2 + np.array([l[0]*np.cos(Q[0]+Q[1]+Q[2]),l[0]*np.sin(Q[0]+Q[1]+Q[2])])
           
            JofX = vJq(Q[0:3],l)
            P_dot = np.array(np.matmul(JofX,Q[3:6])).flatten()[0:2]
        
            return np.append(p3,P_dot) # p = [px, py, pdotx, pdoty]
            
        def JofQ(x): #Move signal to joint space
            # x = [ux,uy,q1,q2,q3] -> uq = [uq1,uq2,uq3]
            u = np.append(x[0:2],0)
            theta = x[2:5]
            JofX = vJq(theta,l)
            u_q = np.matmul(JofX.T,u.T)
            return u_q #uq = [uq1,uq2,uq3]
            
        #Nodes
        
        sys = nengo.Node(arm(L,M,dt),size_in = 9) #inputs = [theta,theta_dot,U] -> [0:3,3:6,6:9] 
                                                  #outputs = [theta,theta_dot] -> [0:3,3:6]
        #ref = nengo.Node([0,0.3,0,0]) #ref = [x_ref,y_ref,x_dot_ref, y_dot_ref]
        ref = nengo.Node( lambda t: [0,0,0,0] if t < 20 else [0.3,0,0,0])
        
        #Ensembles
        err = nengo.Ensemble(num_neurons,4,neuron_type = model_neurons) #err = [ex,ey,exdot,eydot]
        con = nengo.Ensemble(num_neurons,5, neuron_type = model_neurons) #con = [ux,uy,q1,q2,q3]
        
        #Connections
        state_feedback = nengo.Connection(sys,sys[0:6],synapse = sim_synapse) #sys[theta,theta_dot] -> sys[theta,theta_dot]
        ref_in = nengo.Connection(ref,err, synapse = sim_synapse) # ref[x_ref,y_ref,x_dot_ref, y_dot_ref] -> err[ex,ey,exdot,eydot]
        feedback = nengo.Connection(sys,err,function = qTx,transform = -1,synapse = sim_synapse) #sys[theta,theta_dot] -(-qTx)-> err[ex,ey,exdot,eydot]
        uxy = nengo.Connection(err,con[0:2],function = Sliding_Mode,synapse = sim_synapse) #err[ex,ey,exdot,eydot] -(Sliding_Mode)-> con[ux,uy]
        control_state = nengo.Connection(sys[0:3],con[2:5],synapse = sim_synapse) #sys[q1,q2,q3] -> con[q1,q2,q3]
        control_input = nengo.Connection(con,sys[6:9],function = JofQ,synapse = sim_synapse) #con[ux,uy,q1,q2,q3] -(JofQ)-> sys[uq1,uq2,uq3]
        #Theta Control
        theta_control = nengo.Connection(sys[3:6], sys[6:9], transform = -1) #sys[theta_dot] -(-1)-> sys[uq1,uq2,uq3]
    
        
        #Probes
        error_probe = nengo.Probe(target = err, attr = 'decoded_output',synapse = 0.1)
        reference_probe = nengo.Probe(target = ref, attr = 'output',synapse = 0.01)
        state_probe = nengo.Probe(target = feedback, attr = 'output', synapse = 0.01)
        joint_probe = nengo.Probe(target = sys, attr = 'output',synapse = 0.01)
        con_probe = nengo.Probe(target = con,attr = 'decoded_output',synapse = 0.01)
        control_input_probe= nengo.Probe(target = control_input, attr = 'output',synapse = 0.01)
    return model, state_probe, reference_probe

    

## Random Adaptive

In [64]:
def create_model_rand_adapt(weight_update_rate):
    model = nengo.Network()
    with model:
        #Functions
        def Sliding_Mode(e): #e = [ex, ey, edotx, edoty] -> u = [ux,uy]
            s = e[0:2] + lam*e[2:4]
            g_x = m[2]
            return (1/g_x)*(ks*s + lam*e[2:4]) #u = [ux,uy]
            
        def qTx(Q): #Takes joint inputs and converts to end effector
            # Q = [q1,q2,q3,qdot1,qdot2,qdot3] -> p = [px, py, pdotx, pdoty]
            
            p0 = np.array([0,0])
            
            p1 = p0 + np.array([l[0]*np.cos(Q[0]),l[0]*np.sin(Q[0])])
            p2 = p1 + np.array([l[0]*np.cos(Q[0]+Q[1]),l[0]*np.sin(Q[0]+Q[1])])
            p3 = p2 + np.array([l[0]*np.cos(Q[0]+Q[1]+Q[2]),l[0]*np.sin(Q[0]+Q[1]+Q[2])])
           
            JofX = vJq(Q[0:3],l)
            P_dot = np.array(np.matmul(JofX,Q[3:6])).flatten()[0:2]
        
            return np.append(p3,P_dot) # p = [px, py, pdotx, pdoty]
            
        def JofQ(x): #Move signal to joint space
            # x = [ux,uy,q1,q2,q3] -> uq = [uq1,uq2,uq3]
            u = np.append(x[0:2],0)
            theta = x[2:5]
            JofX = vJq(theta,l)
            u_q = np.matmul(JofX.T,u.T)
            return u_q #uq = [uq1,uq2,uq3]
        #Nodes
        
        sys = nengo.Node(arm(L,M,dt),size_in = 9) #inputs = [theta,theta_dot,U] -> [0:3,3:6,6:9] 
                                                  #outputs = [theta,theta_dot] -> [0:3,3:6]
        #ref = nengo.Node([0,0.3,0,0]) #ref = [x_ref,y_ref,x_dot_ref, y_dot_ref]
        ref = nengo.Node( lambda t: [0,0,0,0] if t < 20 else [0.3,0,0,0])
        
        #Ensembles
        err = nengo.Ensemble(num_neurons,4,neuron_type = nengo.Direct()) #err = [ex,ey,exdot,eydot]
        con = nengo.Ensemble(num_neurons,5, neuron_type = nengo.Direct()) #con = [ux,uy,q1,q2,q3]
        
        #Connections
        state_feedback = nengo.Connection(sys,sys[0:6],synapse = sim_synapse) #sys[theta,theta_dot] -> sys[theta,theta_dot]
        ref_in = nengo.Connection(ref,err, synapse = sim_synapse) # ref[x_ref,y_ref,x_dot_ref, y_dot_ref] -> err[ex,ey,exdot,eydot]
        feedback = nengo.Connection(sys,err,function = qTx,transform = -1,synapse = sim_synapse) #sys[theta,theta_dot] -(-qTx)-> err[ex,ey,exdot,eydot]
        uxy = nengo.Connection(err,con[0:2],function = Sliding_Mode,synapse = sim_synapse) #err[ex,ey,exdot,eydot] -(Sliding_Mode)-> con[ux,uy]
        control_state = nengo.Connection(sys[0:3],con[2:5],synapse = sim_synapse) #sys[q1,q2,q3] -> con[q1,q2,q3]
        control_input = nengo.Connection(con,sys[6:9],function = JofQ,synapse = sim_synapse) #con[ux,uy,q1,q2,q3] -(JofQ)-> sys[uq1,uq2,uq3]
        #Theta Control
        theta_control = nengo.Connection(sys[3:6], sys[6:9], transform = -1) #sys[theta_dot] -(-1)-> sys[uq1,uq2,uq3]
    
        #Adaptive
        adaptive_basis = nengo.Ensemble(1000,4, neuron_type = model_neurons)
        adaptive_activation = nengo.Connection(sys,adaptive_basis,function = qTx)
        adaptive_weights = nengo.Connection(adaptive_basis,con[0:2], transform=np.zeros((2,4)), learning_rule_type=nengo.PES(learning_rate=weight_update_rate))
        learing_rule = nengo.Connection(err,adaptive_weights.learning_rule,transform = -1, function = Sliding_Mode)    
        
        #Probes
        error_probe = nengo.Probe(target = err, attr = 'decoded_output',synapse = 0.01)
        reference_probe = nengo.Probe(target = ref, attr = 'output',synapse = 0.01)
        state_probe = nengo.Probe(target = feedback, attr = 'output', synapse = 0.01)
        joint_probe = nengo.Probe(target = sys, attr = 'output',synapse = 0.01)
        con_probe = nengo.Probe(target = con,attr = 'decoded_output',synapse = 0.01)
        control_input_probe= nengo.Probe(target = control_input, attr = 'output',synapse = 0.01)
    return model, state_probe, reference_probe

## Project Basis Adaptive

In [68]:
#For selected basis
#Chooses encoders on D+1 hypersphere
def projection(X): 
    n = len(X) + 1
    X_proj = np.zeros(n)
    X_proj[-1] =  np.prod(np.sin(X))
    for i in range(n-2, -1, -1):
        X_proj[i] = np.prod(np.sin(X[:i])) * np.cos(X[i])
    return X_proj
def selected_encoders(dim,num_neurons):
    X = np.zeros((dim,num_neurons))
    for d in range(dim):
        X[d,:] = np.linspace(-np.pi,np.pi,num_neurons)
    X_proj = np.zeros((dim+1,num_neurons))
    for n in range(num_neurons):
        X_proj[:,n] = projection(X[:,n])
    return np.transpose(X_proj)
def sparsity_to_x_intercept(d, p):
    sign = 1
    if p > 0.5:
        p = 1.0 - p
        sign = -1
    return sign * np.sqrt(1-scipy.special.betaincinv((d-1)/2.0, 0.5, 2*p))
    
def create_model_proj_adapt(weight_update_rate,rho_specified,gain):
    model = nengo.Network()
    with model:
        #Functions
        def Sliding_Mode(e): #e = [ex, ey, edotx, edoty] -> u = [ux,uy]
            s = e[0:2] + lam*e[2:4]
            g_x = m[2]
            return (1/g_x)*(ks*s + lam*e[2:4]) #u = [ux,uy]
            
        def qTx(Q): #Takes joint inputs and converts to end effector
            # Q = [q1,q2,q3,qdot1,qdot2,qdot3] -> p = [px, py, pdotx, pdoty]
            
            p0 = np.array([0,0])
            
            p1 = p0 + np.array([l[0]*np.cos(Q[0]),l[0]*np.sin(Q[0])])
            p2 = p1 + np.array([l[0]*np.cos(Q[0]+Q[1]),l[0]*np.sin(Q[0]+Q[1])])
            p3 = p2 + np.array([l[0]*np.cos(Q[0]+Q[1]+Q[2]),l[0]*np.sin(Q[0]+Q[1]+Q[2])])
           
            JofX = vJq(Q[0:3],l)
            P_dot = np.array(np.matmul(JofX,Q[3:6])).flatten()[0:2]
        
            return np.append(p3,P_dot) # p = [px, py, pdotx, pdoty]
            
        def JofQ(x): #Move signal to joint space
            # x = [ux,uy,q1,q2,q3] -> uq = [uq1,uq2,uq3]
            u = np.append(x[0:2],0)
            theta = x[2:5]
            JofX = vJq(theta,l)
            u_q = np.matmul(JofX.T,u.T)
            return u_q #uq = [uq1,uq2,uq3]
            
        def input_projection(X): #Projects states into D+1 hypersphere (Selected basis only)
            X = qTx(X)
            X = np.divide(X-x_mean,np.asarray(limits))
            n = len(X) + 1
            X_proj = np.zeros(n)
            X_proj[-1] =  np.prod(np.sin(X))
            for i in range(n-2, -1, -1):
                X_proj[i] = np.prod(np.sin(X[:i])) * np.cos(X[i])
            return X_proj
        #Nodes
        
        sys = nengo.Node(arm(L,M,dt),size_in = 9) #inputs = [theta,theta_dot,U] -> [0:3,3:6,6:9] 
                                                  #outputs = [theta,theta_dot] -> [0:3,3:6]
        #ref = nengo.Node([0,0.3,0,0]) #ref = [x_ref,y_ref,x_dot_ref, y_dot_ref]
        ref = nengo.Node( lambda t: [0,0,0,0] if t < 20 else [0.3,0,0,0])
        
        #Ensembles
        err = nengo.Ensemble(num_neurons,4,neuron_type = nengo.Direct()) #err = [ex,ey,exdot,eydot]
        con = nengo.Ensemble(num_neurons,5, neuron_type = nengo.Direct()) #con = [ux,uy,q1,q2,q3]
        
        #Connections
        state_feedback = nengo.Connection(sys,sys[0:6],synapse = sim_synapse) #sys[theta,theta_dot] -> sys[theta,theta_dot]
        ref_in = nengo.Connection(ref,err, synapse = sim_synapse) # ref[x_ref,y_ref,x_dot_ref, y_dot_ref] -> err[ex,ey,exdot,eydot]
        feedback = nengo.Connection(sys,err,function = qTx,transform = -1,synapse = sim_synapse) #sys[theta,theta_dot] -(-qTx)-> err[ex,ey,exdot,eydot]
        uxy = nengo.Connection(err,con[0:2],function = Sliding_Mode,synapse = sim_synapse) #err[ex,ey,exdot,eydot] -(Sliding_Mode)-> con[ux,uy]
        control_state = nengo.Connection(sys[0:3],con[2:5],synapse = sim_synapse) #sys[q1,q2,q3] -> con[q1,q2,q3]
        control_input = nengo.Connection(con,sys[6:9],function = JofQ,synapse = sim_synapse) #con[ux,uy,q1,q2,q3] -(JofQ)-> sys[uq1,uq2,uq3]
        #Theta Control
        theta_control = nengo.Connection(sys[3:6], sys[6:9], transform = -1) #sys[theta_dot] -(-1)-> sys[uq1,uq2,uq3]
    
        #Adaptive
        xi = - sparsity_to_x_intercept( d = 4, p = rho_specified )
        adaptive_basis = nengo.Ensemble(num_neurons,5,
                                        encoders = selected_encoders(4,num_neurons),
                                        gain = gain*np.ones(num_neurons),
                                        bias = np.zeros(num_neurons)+xi,
                                        neuron_type = model_neurons) 
        adaptive_weights = nengo.Connection(adaptive_basis,con[0:2], transform=np.zeros((2,5)), learning_rule_type=nengo.PES(learning_rate=weight_update_rate))
        learning_sig = nengo.Connection(err,adaptive_weights.learning_rule,transform = -1, function = Sliding_Mode) 
        adaptive_input = nengo.Connection(sys,adaptive_basis, function = input_projection) #(sys,adaptive_basis,function = qTx) 
        
        #Probes
        error_probe = nengo.Probe(target = err, attr = 'decoded_output',synapse = 0.01)
        reference_probe = nengo.Probe(target = ref, attr = 'output',synapse = 0.01)
        state_probe = nengo.Probe(target = feedback, attr = 'output', synapse = 0.01)
        joint_probe = nengo.Probe(target = sys, attr = 'output',synapse = 0.01)
        con_probe = nengo.Probe(target = con,attr = 'decoded_output',synapse = 0.01)
        control_input_probe= nengo.Probe(target = control_input, attr = 'output',synapse = 0.01)
    return model, state_probe, reference_probe

## SSP Basis Adaptive

In [73]:


def make_unitary_matrix_fourier( ssp_dim, domain_dim, eps=1e-3, rng = np.random, psd_sampling = 'uniform' ):
    if psd_sampling == 'gaussian':
        # gaussian kernel
        a = rng.normal( loc = 0., scale = 1., size = ( (ssp_dim - 1)//2, domain_dim) )
        phi = np.pi * (eps + a * (1 - 2 * eps))
    
    elif psd_sampling == 'uniform':
        # sinc kernel
        a = rng.rand( (ssp_dim - 1)//2, domain_dim )
        sign = rng.choice((-1, +1), size=np.shape(a) )
        phi = sign * np.pi * (eps + a * (1 - 2 * eps))
    
    fv = np.zeros( (ssp_dim,domain_dim), dtype='complex64')
    fv[0,:] = 1

    fv[1:(ssp_dim + 1) // 2,:] = phi
    fv[-1:ssp_dim // 2:-1,:] = -fv[1:(ssp_dim + 1) // 2,:]
    
    if ssp_dim % 2 == 0:
        fv[ssp_dim // 2,:] = 1

    return fv

class SSPEncoder:
    def __init__(self, phase_matrix, length_scale):
        '''
        Represents a domain using spatial semantic pointers.

        Parameters:
        -----------

        phase_matrix : np.ndarray
            A ssp_dim x domain_dim ndarray representing the frequency 
            components of the SSP representation.

        length_scale : float or np.ndarray
            Scales values before encoding.
        '''
        self.phase_matrix = phase_matrix

        self.domain_dim = self.phase_matrix.shape[1]
        self.ssp_dim = self.phase_matrix.shape[0]
        self.update_lengthscale(length_scale)

    def update_lengthscale(self, scale):
        '''
        Changes the lengthscale being used in the encoding.
        '''
        if not isinstance(scale, np.ndarray) or scale.size == 1:
            self.length_scale = scale * np.ones((self.domain_dim,))
        else:
            assert scale.size == self.domain_dim
            self.length_scale = scale
        assert self.length_scale.size == self.domain_dim
    
    def encode(self,x):
        '''
        Transforms input data into an SSP representation.

        Parameters:
        -----------
        x : np.ndarray
            A (num_samples, domain_dim) array representing data to be encoded.

        Returns:
        --------
        data : np.ndarray
            A (num_samples, ssp_dim) array of the ssp representation of the data
            
        '''
        
        x = np.atleast_2d(x)
        ls_mat = np.atleast_2d(np.diag(1/self.length_scale.flatten()))
        
        assert ls_mat.shape == (self.domain_dim, self.domain_dim), f'Expected Len Scale mat with dimensions {(self.domain_dim, self.domain_dim)}, got {ls_mat.shape}'
        scaled_x = x @ ls_mat
        data = np.fft.ifft( np.exp( 1.j * self.phase_matrix @ scaled_x.T), axis=0 ).real
        
        return data.T

def RandomSSPSpace(domain_dim, ssp_dim, length_scale = None, 
                   rng = np.random.default_rng(), psd_sampling = 'uniform' ):
    
    phase_matrix = make_unitary_matrix_fourier(ssp_dim,domain_dim, psd_sampling = psd_sampling )

    if isinstance(length_scale,float):
        length_scale = np.array( np.tile(length_scale,domain_dim) )
    else:
        length_scale = np.array( length_scale )
    return SSPEncoder(phase_matrix, length_scale=length_scale)
    
    
def encode_rff( x, n_features, length_scale, kernel = 'gaussian', coefficient = 1., n_functions = 1):
    # print(type(x))
    
    # dimensionality of feature space
    x_dim = x.shape[-1]
    omega_shape = (n_functions, n_features, x_dim)
    
    if kernel == 'gaussian':
        omega = np.random.normal( size = omega_shape )
        
    # Scale omegas by lengthscale
    omega = omega / length_scale

    phi = np.random.uniform( low = 0., high = (2 * np.pi), size = (n_functions, n_features, 1) )
    
    features = np.cos( np.einsum('sfd, nd -> sfn', omega, x) + phi)
    features = (2 / n_features) ** 0.5 * features * coefficient

    return features[0,:,:].T




def create_model_SSP_adapt(learning_rate, rho_specified, gain):
    model = nengo.Network()
    with model:
        #Functions
        def Sliding_Mode(e): #e = [ex, ey, edotx, edoty] -> u = [ux,uy]
            s = e[0:2] + lam*e[2:4]
            g_x = m[2]
            return (1/g_x)*(ks*s + lam*e[2:4]) #u = [ux,uy]
            
        def qTx(Q): #Takes joint inputs and converts to end effector
            # Q = [q1,q2,q3,qdot1,qdot2,qdot3] -> p = [px, py, pdotx, pdoty]
            
            p0 = np.array([0,0])
            
            p1 = p0 + np.array([l[0]*np.cos(Q[0]),l[0]*np.sin(Q[0])])
            p2 = p1 + np.array([l[0]*np.cos(Q[0]+Q[1]),l[0]*np.sin(Q[0]+Q[1])])
            p3 = p2 + np.array([l[0]*np.cos(Q[0]+Q[1]+Q[2]),l[0]*np.sin(Q[0]+Q[1]+Q[2])])
           
            JofX = vJq(Q[0:3],l)
            P_dot = np.array(np.matmul(JofX,Q[3:6])).flatten()[0:2]
        
            return np.append(p3,P_dot) # p = [px, py, pdotx, pdoty]
            
        def JofQ(x): #Move signal to joint space
            # x = [ux,uy,q1,q2,q3] -> uq = [uq1,uq2,uq3]
            u = np.append(x[0:2],0)
            theta = x[2:5]
            JofX = vJq(theta,l)
            u_q = np.matmul(JofX.T,u.T)
            return u_q #uq = [uq1,uq2,uq3]
            
        def input_projection(X): #Projects states into D+1 hypersphere (Selected basis only)
            X = qTx(X)
            X = np.divide(X-x_mean,np.asarray(limits))
            n = len(X) + 1
            X_proj = np.zeros(n)
            X_proj[-1] =  np.prod(np.sin(X))
            for i in range(n-2, -1, -1):
                X_proj[i] = np.prod(np.sin(X[:i])) * np.cos(X[i])
            return X_proj
        #Nodes
        
        sys = nengo.Node(arm(L,M,dt),size_in = 9) #inputs = [theta,theta_dot,U] -> [0:3,3:6,6:9] 
                                                  #outputs = [theta,theta_dot] -> [0:3,3:6]
        #ref = nengo.Node([0,0.3,0,0]) #ref = [x_ref,y_ref,x_dot_ref, y_dot_ref]
        ref = nengo.Node( lambda t: [0,0,0,0] if t < 20 else [0.3,0,0,0])
        
        #Ensembles
        err = nengo.Ensemble(num_neurons,4, neuron_type = nengo.Direct()) #err = [ex,ey,exdot,eydot]
        con = nengo.Ensemble(num_neurons,5, neuron_type =  nengo.Direct()) #con = [ux,uy,q1,q2,q3]
        
        #Connections
        state_feedback = nengo.Connection(sys,sys[0:6],synapse = sim_synapse) #sys[theta,theta_dot] -> sys[theta,theta_dot]
        ref_in = nengo.Connection(ref,err, synapse = sim_synapse) # ref[x_ref,y_ref,x_dot_ref, y_dot_ref] -> err[ex,ey,exdot,eydot]
        feedback = nengo.Connection(sys,err,function = qTx,transform = -1,synapse = sim_synapse) #sys[theta,theta_dot] -(-qTx)-> err[ex,ey,exdot,eydot]
        uxy = nengo.Connection(err,con[0:2],function = Sliding_Mode,synapse = sim_synapse) #err[ex,ey,exdot,eydot] -(Sliding_Mode)-> con[ux,uy]
        control_state = nengo.Connection(sys[0:3],con[2:5],synapse = sim_synapse) #sys[q1,q2,q3] -> con[q1,q2,q3]
        control_input = nengo.Connection(con,sys[6:9],function = JofQ,synapse = sim_synapse) #con[ux,uy,q1,q2,q3] -(JofQ)-> sys[uq1,uq2,uq3]
        #Theta Control
        theta_control = nengo.Connection(sys[3:6], sys[6:9], transform = -1) #sys[theta_dot] -(-1)-> sys[uq1,uq2,uq3]
    
        #Adaptive
        ssp_embedding = RandomSSPSpace( domain_dim = 4, 
                                       ssp_dim = ssp_dim, 
                                       length_scale = length_scale, 
                                       psd_sampling = psd_sampling )
        xi = - sparsity_to_x_intercept( d = ssp_dim, p = rho_specified )
        
        def encode_ssp(t,x):
            #print(x)
            return ssp_embedding.encode(x).flatten()
    
        if encoders_type == 'random':
            encoders = nengo.dists.UniformHypersphere(surface=True).sample(num_neurons, ssp_dim)
        elif encoders_type == 'place-cells':
            e_xs = np.random.uniform(low=domain_ranges_[:,0],high=domain_ranges_[:,1],size=(num_neurons,ssp_embedding.domain_dim))
            encoders = ssp_embedding.encode(e_xs)
    
        # create ensemble; random encoders for now
        adaptive_basis = nengo.Ensemble(n_neurons = num_neurons, 
                                  dimensions = ssp_embedding.ssp_dim,
                                  gain = gain*np.ones(num_neurons),
                                  bias = np.zeros(num_neurons) + xi,
                                  neuron_type = model_neurons,
                                  encoders = encoders,
                                  normalize_encoders = False,
                                 )
        ssp_node = nengo.Node(encode_ssp,size_in=4)
        nengo.Connection(sys,ssp_node,function = qTx)
        nengo.Connection(ssp_node,adaptive_basis)            
        adaptive_weights = nengo.Connection(adaptive_basis,con[0:2],transform = np.zeros((2,adaptive_basis.dimensions)),learning_rule_type=nengo.PES(learning_rate=weight_update_rate))
        learning_sig = nengo.Connection(err,adaptive_weights.learning_rule,function = Sliding_Mode,transform = -1) 
        
        #Probes
        error_probe = nengo.Probe(target = err, attr = 'decoded_output',synapse = 0.1)
        reference_probe = nengo.Probe(target = ref, attr = 'output',synapse = 0.01)
        state_probe = nengo.Probe(target = feedback, attr = 'output', synapse = 0.01)
        joint_probe = nengo.Probe(target = sys, attr = 'output',synapse = 0.01)
        con_probe = nengo.Probe(target = con,attr = 'decoded_output',synapse = 0.01)
        control_input_probe= nengo.Probe(target = control_input, attr = 'output',synapse = 0.01)
    return model, state_probe, reference_probe